In [74]:
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot

import requests
import logging
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import sqlite3


logging.getLogger().setLevel(logging.INFO)


In [56]:
class ScrapeTrendingView:

    def __init__(self, company_url):

        

        # innitialize and set chrome-webdriver options
        chrome_options = Options()
        chrome_options.add_argument("--start-maximized")
        # self.chrome_options.add_argument("--window-size=1000,1080")
        # chrome_options.add_argument("--headless")

        self.driver = webdriver.Chrome(
            "G:\My Drive\Investing\Programming\chromedriver.exe", options=chrome_options)
        self.driver.implicitly_wait(2)
        self.time_sleep = 2
        # self.driver.maximize_window()

        self.scrapeIncomeStatement(company_url=company_url)
        self.scrapeBalanceSheet(company_url=company_url)
        self.scrapeCashFlow(company_url=company_url)
        self.scrapeStatistics(company_url=company_url)
        self.scrapeAdditionalData(company_url = company_url)

        self.driver.close()

    def scrapeIncomeStatement(self, company_url):

        url = "https://www.tradingview.com/symbols/" + \
            company_url + "/financials-income-statement/?selected="

        self.driver.get(url)
        time.sleep(self.time_sleep)

        self.switch_annual_data()
        self.close_cookies_popup()
        self.wait_page_to_load()

        # expand income-statement collapsed rows
        i = 0
        while True:
            i = i+1
            if i > 20:
                logging.info(f'Break While loop i={i}')
                break
            try:
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                print(f'i={i}, all rows expanded')
                break

        # scrape the data
        output = self.scrape_the_data()

        self.income_statement = self.scraped_data_to_dataframe(output=output)

    def scrapeBalanceSheet(self, company_url):
        logging.info('Start Balance Sheet Scrape')
        url = "https://www.tradingview.com/symbols/" + \
            company_url + "/financials-balance-sheet/?selected="
        self.driver.get(url)
        time.sleep(self.time_sleep)

        self.switch_annual_data()
        self.wait_page_to_load()
        # self.close_cookies_popup()

        # expand balance-sheet collapsed-rows level-1
        i = 0
        while True:
            # logging.info('Start Expanding Balance Sheet Rows Level-1')
            i = i+1
            if i > 20:
                logging.info(f'Break While loop i={i}')
                break
            try:
                # expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k hasChanges-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                # logging.info('End Expanding Balance Sheet Rows Level-1')
                break

         # expand balance-sheet collapsed-rows level-2
        i = 0
        while True:
            # logging.info('Start Expanding Balance Sheet Rows Level-2')
            i = i+1
            if i > 20:
                logging.info(f'Break While loop i={i}')
                break
            try:
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                # logging.info('End Expanding Balance Sheet Rows Level-2')
                break

        # scrape the data
        output = self.scrape_the_data()
        self.balanse_sheet = self.scraped_data_to_dataframe(output=output)

    def scrapeCashFlow(self, company_url):

        logging.info('Start CashFlow Scrape')
        url = "https://www.tradingview.com/symbols/" + \
            company_url + "/financials-cash-flow/?selected="

        self.driver.get(url)
        time.sleep(self.time_sleep)

        self.switch_annual_data()
        self.wait_page_to_load()
        # self.close_cookies_popup()

        # expand cash-flow collapsed-rows level-1
        i = 0
        while True:
            # logging.info('Start Expanding CashFlow Rows Level-1')
            i = i+1
            if i > 20:
                logging.info(f'Break While loop i={i}')
                break
            try:
                # expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k hasChanges-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                logging.info('End Expanding CashFlow Rows Level-1')
                break

        # expand cash-flow collapsed-rows level-2
        i = 0
        while True:
            # logging.info('Start Expanding CashFlow Rows Level-2')
            i = i+1
            if i > 20:
                logging.info(f'Break While loop i={i}')
                break
            try:
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                # logging.info('End Expanding CashFlow Rows Level-2')
                break

        # scrape the data
        output = self.scrape_the_data()
        self.cashflow_statement = self.scraped_data_to_dataframe(output=output)

    def scrapeStatistics(self, company_url):
        logging.info('Start Statistics Scrape')
        url = "https://www.tradingview.com/symbols/" + company_url + "/financials-statistics-and-ratios/?selected="
        self.driver.get(url)
        time.sleep(self.time_sleep)

        # self.close_cookies_popup()
        self.switch_annual_data()
        self.wait_page_to_load()

        statistics_table_xpath = "//div[@class='container-YOfamMRP']/div"
        statistics_table_rows = self.driver.find_elements_by_xpath(
            statistics_table_xpath)

        # for item in financial_table[1]:
        #     print(item)

        # print(self.financial_table.text)
        print(len(statistics_table_rows))
        output = []

        for item in statistics_table_rows:
            item_list = item.text.splitlines()
            output_temp = []

            # skip non-data items like Key stats, Profitability ratios, Liquidity ratios, Solvency ratios
            if len(item_list) == 1:
                continue
            else:
                for i in range(len(item_list)):

                    output_temp.append(item_list[i].replace(
                        '\u202a', '').replace('\u202c', ''))
                    # print(temp[i])

                # print(type(temp), len(temp))
                # print(temp)
                output.append(output_temp)

        # for item in output:
        #     print(len(item),item)
        #     pass

        self.statistics = self.scraped_data_to_dataframe(output=output)


    def scrapeAdditionalData(self, company_url):
        url = "https://www.tradingview.com/symbols/" + company_url + "/forecast/"
        self.driver.get(url)
        time.sleep(self.time_sleep)
        self.wait_page_to_load()

        company_name_css = "h2[class='tv-symbol-header__first-line']"
        company_name_element = self.driver.find_element(By.CSS_SELECTOR, company_name_css)
        self.company_name = company_name_element.text
        # print(self.company_name)
        
    
    def close_cookies_popup(self):
        cookie_button_xpath = "//button[@class='acceptAll-WvyPjcpY button-OvB35Th_ size-xsmall-OvB35Th_ color-brand-OvB35Th_ variant-primary-OvB35Th_']"
        cookie_button_element = self.driver.find_element_by_xpath(
            cookie_button_xpath)
        cookie_button_element.click()

    def switch_annual_data(self):
        annual_button_xpath = "//button[@id='FY']"
        annual_button_element = self.driver.find_element_by_xpath(
            annual_button_xpath)
        annual_button_element.click()

    def scraped_data_to_dataframe(self, output):
        output_index = []
        output_values = []
        output_colums = output[0][1:]
        self.currency = output[0][0].replace('Currency: ', '')
        # print(self.currency)

        for i in range(1, len(output)):
            output_index.append(output[i][0])
            output_values.append(output[i][1:])

        # apply neccessery correction to fix the values-data
        output_values = self.fix_data_values(input_data=output_values)
        df = pd.DataFrame(output_values, columns=output_colums,
                          index=output_index)  # add scraped data to dataframe
        return df

    def scrape_the_data(self):

        # financial_table_xpath = "//div[@class='container-YOfamMRP']"
        financial_table_xpath = "//div[@class='container-YOfamMRP']/div"
        financial_table_rows = self.driver.find_elements_by_xpath(
            financial_table_xpath)
        print(len(financial_table_rows))

        output = []
        number_of_periods = len(financial_table_rows[0].text.splitlines())

        for item in financial_table_rows:
            item_list = item.text.splitlines()
            output_temp = []

            if len(item_list) == number_of_periods:  # rows without YOY-grow

                for i in range(len(item_list)):
                    output_temp.append(item_list[i].replace(
                        '\u202a', '').replace('\u202c', ''))
            else:  # rows with YOY-grow
                if 'YoY growth' in item_list:  # Quarterly
                    for i in range(0, len(item_list), 2):  # skip YOY-grow row
                        output_temp.append(item_list[i].replace(
                            '\u202a', '').replace('\u202c', ''))
                else:  # Anual report
                    output_temp.append(item_list[0])
                    for i in range(1, len(item_list), 2):  # skip YOY-grow row
                        output_temp.append(item_list[i].replace(
                            '\u202a', '').replace('\u202c', ''))

            output.append(output_temp)

        return output


    def fix_data_values(self, input_data):
        output = []

        for row in input_data:
            output_row = []
            for item in row:
                # print(f'item={item}')

                if '−' in item:  # convert minus sign to real minus, for some reason the sign is not recognized as minus
                    item = item.replace('−', '-')

                if 'T' in item:  # convert Trillion-values to numeric
                    item = item.replace('T', '')
                    item = float(item)
                    item = item*1000000000000
                    # item = int(item)

                elif 'B' in item:  # convert Billion-values to numeric
                    item = item.replace('B', '')
                    item = float(item)
                    item = item*1000000000
                    # item = int(item)

                elif 'M' in item:  # convert Milion-values to numeric
                    item = item.replace('M', '')
                    item = float(item)
                    item = item*1000000
                    # item = int(item)

                elif 'K' in item:  # convert Thousants-values to numeric
                    item = item.replace('K', '')
                    item = float(item)
                    item = item*1000
                    # item = int(item)

                if isinstance(item, str):  # if item is not integer (0.00, ---, -)

                    if '—' in item:  # set value to None
                        item = None

                    elif '.' in item:  # convert value to float
                        item = float(item)

                if self.currency != 'USD':  # convert values to USD

                    if self.currency == 'KRW':
                        self.multiplier = 0.000700680009950

                    # check if item is int or float
                    if isinstance(item, float) or isinstance(item, int):
                        item = item*self.multiplier

                output_row.append(item)

            output.append(output_row)

        return output

    def wait_page_to_load(self):
        #wait the following items to be loaded before proceeding forward with the scraping procedures
        price_css = "div[class='tv-symbol-price-quote__value js-symbol-last']"
        price_element = self.driver.find_element(By.CSS_SELECTOR, price_css)

        currency_symbol_css = ".tv-symbol-price-quote__currency.js-symbol-currency"
        currency_symbol_element = self.driver.find_element(By.CSS_SELECTOR, currency_symbol_css)

        prince_increase_css = ".js-symbol-change.tv-symbol-price-quote__change-value"
        prince_increase_element = self.driver.find_element(By.CSS_SELECTOR, prince_increase_css)

        prince_increase_percent_css = ".js-symbol-change-pt.tv-symbol-price-quote__change-value"
        prince_increase_percent_element = self.driver.find_element(By.CSS_SELECTOR, prince_increase_percent_css)

        

# run_scraper = ScrapeTrendingView()
# run_scraper.driver.close()


In [96]:
# apple = ScrapeTrendingView(company_url='NASDAQ-AAPL')
# tesla = ScrapeTrendingView(company_url='NASDAQ-TSLA')
# berkshire = ScrapeTrendingView(company_url='NYSE-BRK.A')
companies = [apple, tesla, berkshire]

# apple.income_statement
# apple.balanse_sheet
# apple.cashflow_statement
# apple.statistics

# tesla.income_statement
# tesla.balanse_sheet
# tesla.cashflow_statement
# tesla.statistics

# berkshire.income_statement
# berkshire.balanse_sheet
# berkshire.cashflow_statement
# berkshire.statistics

DataBase.AddToDatabase(data=apple)


Storring Apple_Income_Statement in tradingview_database.db
Storring Apple_Balance_Sheet in tradingview_database.db
Storring Apple_Cashflow_Statement in tradingview_database.db
Storring Apple_Ratios in tradingview_database.db


In [95]:
class DataBase():
    
    database_file = 'tradingview_database.db'
    
    @classmethod
    def AddToDatabase(cls, data):
        
        __class__.StoreInDatabase(data=data.income_statement, name='Apple_Income_Statement')
        __class__.StoreInDatabase(data=data.balanse_sheet, name='Apple_Balance_Sheet')
        __class__.StoreInDatabase(data=data.cashflow_statement, name='Apple_Cashflow_Statement')
        __class__.StoreInDatabase(data=data.statistics, name='Apple_Ratios')

    @classmethod
    def StoreInDatabase(cls, data, name):
        print(f'Storring {name} in {__class__.database_file}')
        conn = sqlite3.connect(__class__.database_file) #connect to sqlite3-database
        data.to_sql(name=name, con=conn, if_exists='replace',  index=True) #store dataframe to sqlite-database
        conn.close()

    @classmethod
    def ReadFromDatabase(self):
        pass


In [55]:
#comparison visials of two or more companies

# df_apple = apple.statistics.transpose()
# fig_apple = px.line(df_apple, x=df_apple.index, y=[
#                     'Number of employees'], title='Apple', markers=True)
# fig_apple.show()

# df_tesla = tesla.statistics.transpose()
# fig_tesla = px.line(df_tesla, x=df_tesla.index, y=[
#                     'Number of employees'], title='Tesla', markers=True)
# fig_tesla.show()

# apple.statistics.index

# fig = px.line(x=apple.statistics.columns, y=apple.statistics.loc['Number of employees'], title='', markers=True)
# fig.show()

# dict for the dataframes and their names

# plot the data
fig = go.Figure()

for company in companies:
    i=8
    company_name = company.company_name
    parameter_name = company.statistics.index[i]
    columns = company.statistics.columns
    rows = company.statistics.iloc[i]
    table_name = f"{company_name}/{parameter_name}"
    fig = fig.add_trace(go.Scatter(x = columns, y= rows, name= table_name))

fig.update_xaxes(categoryorder='category ascending')  # sort X-axis (when X-axis of different companies contains different ranges i.e. 2015-2021, 2016-2022)
fig.update_traces(mode="markers+lines", hovertemplate=None) #enable hover-mode, interactively display values on the graph when pointed with mouse
fig.update_layout(hovermode="x", hoverlabel_namelength=-1) #display the full parameter name

fig.show()


# fig =  px.line(x = tesla.statistics.columns, y = tesla.statistics.iloc[1])
# fig = px.line(x =apple.statistics.columns, y = apple.statistics.iloc[1])
# fig.show()

# apple.statistics.iloc[2]


In [ ]:
# plotting the data
plt.figure(figsize=(20, 8))  # set figure size


# tesla_data.financial_ratios_quarter.loc['Current Ratio'].plot()
# apple_data.financial_ratios_quarter.loc['Current Ratio'].plot()

# print(run_scraper.income_statement.index[0])
# param_name = run_scraper.income_statement.index[0] #get parameter name from dataframe columns

i = 1
apple.statistics.iloc[i].plot()
tesla.statistics.iloc[i].plot()

# print(apple.statistics.iloc[i])
# print(tesla.statistics.iloc[i])
# run_scraper.income_statement.loc['Interest expense on debt'].plot()
# plt.plot()

# invert x-axis (2009 on the left, 2022 on the right)
# ax = plt.gca()
# ax.invert_xaxis()
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# df = px.data.gapminder().query("country=='Canada'")
fig = px.line(x=run_scraper.income_statement.columns,
              y=[run_scraper.income_statement.loc['Total revenue'],
                 run_scraper.income_statement.loc['Net income']],
              title='Total Revenue')
fig.show()


In [ ]:
run_scraper.income_statement.transpose()


In [15]:
# income statement visuals
df_income_statement = run_scraper.income_statement.transpose()

fig_revenue = px.line(df_income_statement, x=df_income_statement.index, y=['Total revenue', 'Cost of goods sold', 'Gross profit'],
                      title='Total Revenue - Cost Of Goods Sold = Gross Profit', markers=True)
fig_revenue.show()

fig_operating_income = px.line(df_income_statement, x=df_income_statement.index, y=['Operating expenses (excl. COGS)', 'Operating income', 'Gross profit'],
                               title='Gross Profit - Operating Expenses = Operating Income', markers=True)
fig_operating_income.show()

fig_pretax_income = px.line(df_income_statement, x=df_income_statement.index, y=['Operating income', 'Non-operating income, total', 'Pretax income'],
                            title='Operating Income + Non Operating Income = Pretax Income', markers=True)
fig_pretax_income.show()

fig_discontinued_operations = px.line(df_income_statement, x=df_income_statement.index, y=['Non-controlling/minority interest', 'After tax other income/expense',
                                                                                           'Net income before discontinued operations', 'Discontinued operations', ],
                                      title='Net income before discontinued operations', markers=True)
fig_discontinued_operations.show()

fig_net_income = px.line(df_income_statement, x=df_income_statement.index, y=['Pretax income', 'Taxes', 'Net income'],
                         title='Pretax Income - Taxes = Net Income', markers=True)
fig_net_income.show()

fig_diluted_net_income = px.line(df_income_statement, x=df_income_statement.index, y=['Dilution adjustment', 'Preferred dividends', 'Diluted net income available to common stockholders'],
                                 title='Net Income + Dilution Adjustment - Preferred Dividents = Diluted Net Income', markers=True)
fig_diluted_net_income.show()

fig_eps = px.line(df_income_statement, x=df_income_statement.index, y=['Basic earnings per share (Basic EPS)', 'Diluted earnings per share (Diluted EPS)'],
                  title='Earnings Per Share', markers=True)
fig_eps.show()

fig_shares = px.line(df_income_statement, x=df_income_statement.index, y=['Average basic shares outstanding', 'Diluted shares outstanding'],
                     title='Shares Outstanding', markers=True)
fig_shares.show()

fig_ebit = px.line(df_income_statement, x=df_income_statement.index, y=['EBITDA', 'EBIT'],
                   title='EBIT/EBITDA', markers=True)
fig_ebit.show()

fig_operating_expenses = px.line(df_income_statement, x=df_income_statement.index, y=['Operating expenses (excl. COGS)', 'Cost of goods sold', 'Total operating expenses'],
                                 title='Operating Expenses + Cost of Goods Sold = Total Operating Expenses', markers=True)
fig_operating_expenses.show()

# Index(['Total revenue', 'Cost of goods sold', 'Deprecation and amortization',
#        'Depreciation', 'Amortization of intangibles',
#        'Amortization of deferred charges', 'Other cost of goods sold',
#        'Gross profit', 'Operating expenses (excl. COGS)',
#        'Selling/general/admin expenses, total', 'Research & development',
#        'Selling/general/admin expenses, other',
#        'Other operating expenses, total', 'Operating income',
#        'Non-operating income, total',
#        'Interest expense, net of interest capitalized',
#        'Interest expense on debt', 'Interest capitalized',
#        'Non-operating income, excl. interest expenses',
#        'Non-operating interest income', 'Pretax equity in earnings',
#        'Miscellaneous non-operating expense', 'Unusual income/expense',
#        'Impairments', 'Restructuring charge', 'Legal claim expense',
#        'Unrealized gain/loss', 'Other exceptional charges', 'Pretax income',
#        'Equity in earnings', 'Taxes', 'Income tax, current',
#        'Income tax, current - domestic', 'Income Tax, current - foreign',
#        'Income tax, deferred', 'Income tax, deferred - domestic',
#        'Income tax, deferred - foreign', 'Income Tax Credits',
#        'Non-controlling/minority interest', 'After tax other income/expense',
#        'Net income before discontinued operations', 'Discontinued operations',
#        'Net income', 'Dilution adjustment', 'Preferred dividends',
#        'Diluted net income available to common stockholders',
#        'Basic earnings per share (Basic EPS)',
#        'Diluted earnings per share (Diluted EPS)',
#        'Average basic shares outstanding', 'Diluted shares outstanding',
#        'EBITDA', 'EBIT', 'Total operating expenses'],
#       dtype='object')


In [32]:
# balanse sheet
df_balanse_sheet = run_scraper.balanse_sheet.transpose()

fig_total_assets_liabilities_equity = px.line(df_balanse_sheet, x=df_balanse_sheet.index, y=['Total assets', 'Total liabilities', 'Total equity',  "Total liabilities & shareholders' equities"],
                                              title='Total Assets/Liabilities/Equity', markers=True)
fig_total_assets_liabilities_equity.show()

fig_current_non_current_assets = px.line(df_balanse_sheet, x=df_balanse_sheet.index, y=['Total current assets', 'Total non-current assets'],
                                         title='Total Current/Non-Current Assets', markers=True)
fig_current_non_current_assets.show()

fig_current_non_current_liabilities = px.line(df_balanse_sheet, x=df_balanse_sheet.index, y=['Total current liabilities', 'Total non-current liabilities'],
                                              title='Total Current/Non-Current Liabilities', markers=True)
fig_current_non_current_liabilities.show()


fig_total_assets_liabilities_equity = px.line(df_balanse_sheet, x=df_balanse_sheet.index, y=['Total debt', 'Net debt'],
                                              title='Total/Net Debt', markers=True)
fig_total_assets_liabilities_equity.show()

fig_book_value_per_share = px.line(df_balanse_sheet, x=df_balanse_sheet.index, y=['Book value per share'],
                                   title='Book Value Per Share', markers=True)
fig_book_value_per_share.show()


# Index(['Total assets', 'Total current assets',
#        'Cash and short term investments', 'Cash & equivalents',
#        'Short term investments', 'Total receivables, net',
#        'Accounts receivable - trade, net', 'Accounts receivables, gross',
#        'Bad debt / Doubtful accounts', 'Other receivables', 'Total inventory',
#        'Inventories - work in progress',
#        'Inventories - progress payments & other',
#        'Inventories - finished goods', 'Inventories - raw materials',
#        'Prepaid expenses', 'Other current assets, total',
#        'Total non-current assets', 'Long term investments',
#        'Note receivable - long term',
#        'Investments in unconsolidated subsidiaries', 'Other investments',
#        'Net property/plant/equipment', 'Gross property/plant/equipment',
#        'Property/plant/equipment - Buildings',
#        'Property/plant/equipment - Construction in progress',
#        'Property/plant/equipment - Machinery & equipment',
#        'Property/plant/equipment - Land & improvement',
#        'Property/plant/equipment - Leased property',
#        'Property/plant/equipment - Leases',
#        'Property/plant/equipment - Computer software and equipment',
#        'Property/plant/equipment - Transportation equipment',
#        'Property/plant/equipment - Other', 'Accumulated depreciation, total',
#        'Accumulated depreciation - Buildings',
#        'Accumulated depreciation - Construction in progress',
#        'Accumulated depreciation - Machinery & equipment',
#        'Accumulated depreciation - Land & improvement',
#        'Accumulated depreciation - Leased property',
#        'Accumulated depreciation - Leases',
#        'Accumulated depreciation - Computer software and equipment',
#        'Accumulated depreciation - Transportation equipment',
#        'Accumulated depreciation - Other', 'Deferred tax assets',
#        'Net intangible assets', 'Goodwill, net', 'Goodwill, gross',
#        'Accumulated goodwill amortization', 'Other intangibles, net',
#        'Other intangibles, gross',
#        'Accumulated amortization of other intangibles', 'Deferred charges',
#        'Other long term assets, total', 'Total liabilities',
#        'Total current liabilities', 'Short term debt',
#        'Current portion of LT debt and capital leases',
#        'Short term debt excl. current portion of LT debt', 'Notes payable',
#        'Other short term debt', 'Accounts payable', 'Income tax payable',
#        'Dividends payable', 'Accrued payroll', 'Deferred income, current',
#        'Other current liabilities', 'Total non-current liabilities',
#        'Long term debt', 'Long term debt excl. lease liabilities',
#        'Capital and operating lease obligations',
#        'Capitalized lease obligations', 'Operating lease liabilities',
#        'Provision for risks & charge', 'Deferred tax liabilities',
#        'Deferred income, non-current', 'Other non-current liabilities, total',
#        'Total equity', 'Shareholders' equity', 'Common equity, total',
#        'Preferred stock, carrying value', 'Minority interest',
#        'Total liabilities & shareholders' equities', 'Total debt', 'Net debt',
#        'Book value per share'],
#       dtype='object')


In [14]:
# visualizing cashflow

df_cashflow = run_scraper.cashflow_statement.transpose()

fig_cashflow_operating_investing_financial = px.line(df_cashflow, x=df_cashflow.index, y=['Cash from operating activities', 'Cash from investing activities',
                                                                                          'Cash from financing activities', 'Free cash flow'],
                                                     title='Cashflow From Operating/Investing/Financial Activities', markers=True)

fig_cashflow_operating_investing_financial.update_traces(
    mode="markers+lines", hovertemplate=None)
fig_cashflow_operating_investing_financial.update_layout(
    hovermode="x", hoverlabel_namelength=-1)
fig_cashflow_operating_investing_financial.show()


fig_cashflow_operating_activities = px.line(df_cashflow, x=df_cashflow.index, y=['Cash from operating activities', 'Funds from operations',
                                                                                 'Changes in working capital'],
                                            title='Cash from operating activities + Funds from operations = Cashflow From Operating Activities', markers=True)
fig_cashflow_operating_activities.show()

fig_cashflow_investing_activities = px.line(df_cashflow, x=df_cashflow.index, y=['Cash from investing activities', 'Purchase/sale of business, net',
                                                                                 'Purchase/sale of investments, net', 'Capital expenditures', 'Other investing cash flow items, total'],
                                            title='Purchase/sale of business + Purchase/sale of investments + Capital expenditures + Other investing cash flow items = Cash from investing activities', markers=True)
fig_cashflow_investing_activities.show()

# Index(['Cash from operating activities', 'Funds from operations',
#        'Net income (cash flow)', 'Depreciation & amortization (cash flow)',
#        'Depreciation/depletion', 'Amortization', 'Deferred taxes (cash flow)',
#        'Non-cash items', 'Changes in working capital',
#        'Change in accounts receivable', 'Change in taxes payable',
#        'Change in accounts payable', 'Change in accrued expenses',
#        'Change in inventories', 'Change in other assets/liabilities',
#        'Cash from investing activities', 'Purchase/sale of business, net',
#        'Sale of fixed assets & businesses', 'Purchase/acquisition of business',
#        'Purchase/sale of investments, net', 'Sale/maturity of investments',
#        'Purchase of investments', 'Capital expenditures',
#        'Capital expenditures - fixed assets',
#        'Capital expenditures - other assets',
#        'Other investing cash flow items, total',
#        'Investing activities – other sources',
#        'Investing activities – other uses', 'Cash from financing activities',
#        'Issuance/retirement of stock, net', 'Sale of common & preferred stock',
#        'Repurchase of common & preferred stock',
#        'Issuance/retirement of debt, net',
#        'Issuance/retirement of long term debt', 'Issuance of long term debt',
#        'Reduction of long term debt', 'Issuance/retirement of short term debt',
#        'Issuance/retirement of other debt', 'Total cash dividends paid',
#        'Common dividends paid', 'Preferred dividends paid',
#        'Other financing cash flow items, total',
#        'Financing activities – other sources',
#        'Financing activities – other uses', 'Free cash flow'],
#       dtype='object')


In [14]:
df_statistics = run_scraper.statistics.transpose()

fig_shares_outstanding = px.line(df_statistics, x=df_statistics.index, y=[
                                 'Total common shares outstanding', 'Float shares outstanding'], title='Number of Shares', markers=True)
fig_shares_outstanding.show()

fig_enterprice_values = px.line(df_statistics, x=df_statistics.index, y=[
                                'Enterprise value'], title='Enterprise value', markers=True)
fig_enterprice_values.show()

fig_numer_of_employees_shareholders = px.line(df_statistics, x=df_statistics.index, y=[
                                              'Number of employees', 'Number of shareholders'], title='Number of employees/shareholders', markers=True)
fig_numer_of_employees_shareholders.show()

fig_price_ratios = px.line(df_statistics, x=df_statistics.index, y=['Price to earnings ratio', 'Price to sales ratio', 'Price to cash flow ratio', 'Enterprise value to EBITDA ratio',
                                                                    'Price to book ratio'], title='Price Ratios', markers=True)
fig_price_ratios.show()

fig_return_ratios = px.line(df_statistics, x=df_statistics.index, y=['Return on assets %', 'Return on equity %', 'Return on invested capital %'],
                            title='Return Ratios', markers=True)
fig_return_ratios.show()

fig_margins = px.line(df_statistics, x=df_statistics.index, y=[
                      'Gross margin %', 'EBITDA margin %', 'Net margin %', 'Operating margin %'], title='Margins', markers=True)
fig_margins.show()

fig_dept_ratios = px.line(df_statistics, x=df_statistics.index, y=[
                          'Debt to assets ratio', 'Debt to equity ratio', 'Long term debt to total assets ratio'], title='Dept Ratios', markers=True)
fig_dept_ratios.show()

fig_liquidity_ratios = px.line(df_statistics, x=df_statistics.index, y=[
                               'Quick ratio', 'Current ratio', 'Inventory turnover', 'Asset turnover'], title='Liquidity Ratios', markers=True)
fig_liquidity_ratios.show()
